In [1]:
# !pip install langchain_community
# !pip install langchain_openai
# !pip install pytube
# !pip install gdown
# !pip install yt_dlp
# !pip install python-dotenv
# !pip install faiss-cpu
# !pip install fastembed

In [2]:
# from general_functions import faiss_index
# from miltimodal_rag_preparation_functions import add_screenshot_descriptions_to_segments, segments_to_langchain_documents
# from langchain_community.embeddings import FastEmbedEmbeddings

# # add_screenshot_descriptions_to_segments("data/for_multimodal_rag/video_6f1zm-lNMGc_segments.json", "data/for_multimodal_rag/screenshots_descriptions.json")

# chunks = segments_to_langchain_documents("data/for_multimodal_rag/video_6f1zm-lNMGc_segments.json")

# faiss_index(chunks=chunks, embedding_model=FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5"), index_name="faiss_index_multimodal_rag")


In [3]:
# from general_functions import extract_fenced_text, faiss_index

# vdb = faiss_index(index_name="faiss_index")

# image_results = vdb.similarity_search("", filter={"type": "media"})

# print(image_results)

In [4]:
from ai_agents import bemyapp_agent
from general_functions import faiss_index
from langchain_core.messages import HumanMessage
from langchain_community.utilities import SQLDatabase
import os
from dotenv import load_dotenv
load_dotenv(".env")

current_agent = "text_to_sql_agent"
text2sql_chat_history = []

questions = {
    "text_to_sql_agent": [
        "What was the total profit for each product in the year 2014?",
        "how about 2013?",
        "by country",
    ],
    "switch_to_multimodal_rag_agent": [
        "I have a question about my video",
    ],
    "multimodal_rag_agent": [
        #  can be answered without images
        "What is the origin of Interactive Technical Manuals (ITMs), and how have they evolved since their creation?",
        "How can ITMs benefit technicians and engineers in troubleshooting and maintaining complex equipment?"
        # need images to answer
        "How does the ITM's schematic interface enhance the understanding of complex hydraulic systems?",
        "What kind of information is typically included in a component sheet within the ITM system?",
    ],
    "switch_to_recursive_rag_agent": [
        "I have a question about my contracts",
    ],
    "recursive_rag_agent": [
        "Which companies provide design services for us?",
        "What is the cost of services of CreativeMind?",
        "Give me contacts of CreativeMind",
    ]
}

db = SQLDatabase.from_uri("sqlite:///text_to_sql_example.db")
multimodal_rag_index = faiss_index(index_name="faiss_index_multimodal_rag")
recursive_rag_index = faiss_index(index_name="faiss_index_recursive_rag")


/opt/anaconda3/envs/bemyapp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 23379.62it/s]


In [5]:
while 1:
    question = input(f"Your message to {current_agent}: ")
    print(f"\n\n\n-----------------------\n\nYOU: {question}\n")
    if question == "exit":
        break
    text2sql_chat_history.append(HumanMessage(content=question))
    response = bemyapp_agent(question, current_agent, text2sql_chat_history, multimodal_rag_index, recursive_rag_index, db)
    current_agent = response["next_agent"]
    # Check if the answer contains media references
    if "MEDIA: " in response['answer'] and question!="What is the origin of Interactive Technical Manuals (ITMs), and how have they evolved since their creation?":
        # Split the answer and media part
        answer, media = response['answer'].split("MEDIA: ")
        # Strip any whitespace
        media = media.strip()
        # Split multiple media files if present
        media_files = media.split(", ")
        
        print(f"{current_agent}: {answer}")
        # print(f"DEBUG: {response['results_transcript_string']}")
        # print(f"DEBUG: {response['results_media_string']}")
        
        # Display each media file
        for media_file in media_files:
            media_file = media_file.replace(".png", ".jpeg").strip('"')
            if not media_file:
                continue
            # print(f"DEBUG: trying to embed {media_file}")
            from IPython.display import Image, display
            image_path = f"data/for_multimodal_rag/{media_file}"
            try:
                # Load and display the image
                display(Image(filename=image_path))
            except Exception as e:
                # print(f"Error displaying {media_file}: {str(e)}")
                # If there's an error, try to display using PIL
                try:
                    from PIL import Image as PILImage
                    img = PILImage.open(image_path)
                    display(img)
                except Exception as inner_e:
                    pass
                    # print(f"Error displaying {media_file} with PIL: {str(inner_e)}")
                    # # If both methods fail, provide a more informative error message
                    # print(f"The image file '{media_file}' could not be found or displayed. Please check if the file exists in the 'data/for_multimodal_rag/' directory and ensure the filename is correct.")
    
    elif question=="What is the origin of Interactive Technical Manuals (ITMs), and how have they evolved since their creation?":
        # Remove trailing media string if present
        answer = response['answer'].split("MEDIA: ")[0].strip()
        print(f"{current_agent}: {answer}")
    else:
        # If no media, just print the answer as before
        print(f"{current_agent}: {response['answer']}")
    print("")
    print("")




-----------------------

YOU: total revenue

text_to_sql_agent: The total revenue is $118,726,350.29.

[SQL query:
SELECT SUM(Sales) AS TotalRevenue FROM financials;]





-----------------------

YOU: What is the origin of Interactive Technical Manuals (ITMs), and how have they evolved since their creation?

multimodal_rag_agent: Interactive Technical Manuals (ITMs) originated from military specifications in the late 1980s, shortly after personal computers became mainstream. The military, particularly the Navy, sought a method to capture and electronically store all technical documentation related to their high-value equipment. This led to the creation of a military specification (MIL-SPEC) for Interactive Electronic Technical Manuals. Initially, producing these manuals was quite expensive, but advancements in technology over the years have made it more feasible to store and manage technical information electronically.

Since their inception, ITMs have evolved significantly. They n